## Let's create a bot....

![Newsbot](https://static.vecteezy.com/system/resources/previews/000/518/290/non_2x/ai-artificial-intelligence-technology-robot-cartoon-001-vector.jpg)

During today's class, we're going to learn how to create a Slack bot. You'll be able to take the bots which you've been prototyping (and hopefully starting to build in code) and turn them into Slack bots. In our next class (after Spring Break 🌴), we'll learn how to move these bots off of our laptop and into the cloud (where they can run 24x7, even while we sleep).

To illustrate some simple bot concepts and how we'll integrate with the Slack APIs and libraries, we're going to build a simple bot related to the coronavirus. Once we learn the Slack bot basics, we'll have you move your bot ideas to Slack.

**Ok, let's get started....**

For our simple newsbot 🗞️🤖, we have a few basic **requirements**:

1. We'd like to be able to ask it for the latest statistics on the virus
2. We'd like to be able to ask it for the most recent statistics from different parts of the world
3. We'd like to be able to get the latest news on the coronavirus

We're going to build this bot on Slack but we'd like to easily "port" it to Alexa as well.


[TODO: show interactions]
e.g.:
```
@covidbot confirmed
@covidbot recovered
@covidbot news about coronavirus
```


<img src="https://external-preview.redd.it/5TUMGrd5nfl7nYnA9RosPBR9l1PUMIMeYplMAtE9oNQ.png?width=960&crop=smart&auto=webp&s=bfd95c6af23083aa439eff0b632a40fcd9475ea0" width="800">

### First, let's look at where our bot will get the news...

We're going to use a nice service call the [News API](https://newsapi.org/). The News API is a service which allows us to: "*Get breaking news headlines, and search for articles from over 30,000 news sources and blogs with our news API*". It provides simple searching access to a large number of news services.

**Please** sign up for an API key - you'll need it in the code below: https://newsapi.org/register

The nice folks at News API have written a [python library](https://newsapi.org/docs/client-libraries/python) so that we can easily call their API. Let's install that now:

In [ ]:
!pip install newsapi-python

Now, let's look at how we can call the News API to get the various news we'll need to handle our bot interactions. Remember, we want out bot to be able to fetch:

1. Top stories
2. The latest stories from a given site or domain name.
3. Search for latest stories given a term/topic.


To get the top headlines from the News API, we can do the following:

In [ ]:
# import the NewsApiClient
from newsapi import NewsApiClient

# put your News API key here
NEWSAPI_KEY = '1235c8f5d0b9425eb17d0b8cd12f8881'

newsapi = NewsApiClient(api_key=NEWSAPI_KEY)

# this is how we call the news api to get the top headlines in the US (english)
top_headlines = newsapi.get_top_headlines(language='en', country='us')

What sort of python object are we working with?

In [ ]:
type(top_headlines)

Let's inspect the dictionary a bit:

In [ ]:
top_headlines.keys()

It looks like we have three fields: `status`, `totalResults` and `articles`. Let's print them out and see what they look like:

In [ ]:
top_headlines['status']

In [ ]:
top_headlines['totalResults']

Let's look at `articles` - what type is it?

In [ ]:
type(top_headlines['articles'])

How would we print out the first article?

In [ ]:
# Your Turn: how would we print out the first article?




The article dictionary has a lot of great info: title, description, url, image, published date and "source".

In [ ]:
# Your Turn: loop over the articles, printing out the title, source name and published date




### Now, let's see how we get the top stories for a given news site:

News API has an API called "[everything](https://newsapi.org/docs/endpoints/everything)" which allows us to pass in a domain and get all of the news from that domain/site. Here is how we get the latest news from the Washington Post:

In [ ]:
# top headlines from WaPo
top_headlines = newsapi.get_everything(domains='washingtonpost.com', language='en')

for article in top_headlines['articles']:
    print(article['title'])
    print(article['source']['name'])
    print(article['publishedAt'])
    print('---'*10)

### Last, how do we get the latest articles for a given search query:

Here is how we'd search for top stories for a given query. In this case, let's look for the news on the "coronavirus":

In [ ]:
query = 'coronavirus'

top_headlines = newsapi.get_top_headlines(q=query, language='en')

for article in top_headlines['articles']:
    print(article['title'])
    print(article['source']['name'])
    print(article['publishedAt'])
    print('---'*10)

## Now, let's get to work on our Slack bot

Now that we have the news-side of our bot (mostly) worked out, let's start on our Slack integration.

**First**, let's create our news bot on Slack:

Make sure you are logged into slack.com with your account you're using for this class and head to [api.slack.com](https://api.slack.com/apps?new_granular_bot_app=1) to create your own bot. Fill out the form by using a clever bot name (you can't all use the same name!) and selecting `Computational Journalism 2020` from the Workspace dropdown. If you don't see "Computational Journalism 2020" in the Workspace dropdown menu, you may need to log back in under a different email/account.

<img src="https://raw.githubusercontent.com/computationaljournalism/columbia2020/master/images/bot_create.png" width="500">

**Next**, we need to give our app permissions so that it can post messages to Slack:

Navigate to `OAuth & Permissions` on the sidebar and scroll down to the `Bot Token Scopes` section. `Scopes` give your app permission to do things (for example, post messages). 

Click on `Add an OAth Scope`.

Add the `chat:write` scope to grant your app the permission to post messages in channels it's a member of.

<img src="https://raw.githubusercontent.com/computationaljournalism/columbia2020/master/images/bot_scopes.png" width="500">

**Now**, scroll back up to the top of the `OAuth & Permissions` screen and click on the green `Install App to Workspace` button.

<img src="https://raw.githubusercontent.com/computationaljournalism/columbia2020/master/images/bot_install.png" width="500">

You should see a prompt that looks like the following - click `Allow` so that your bot can be installed into our Computational Journalism Slack "workplace."

<img src="https://raw.githubusercontent.com/computationaljournalism/columbia2020/master/images/bot_allow.png" width="500">

**Important!!**

Copy the `Bot User OAuth Access Token` - this will be our API token! We'll use it in the code below!

<img src="https://raw.githubusercontent.com/computationaljournalism/columbia2020/master/images/bot_token.png" width="600">

**Last...**

As we develop our bot, each of us will create a new channel in Slack that we can use as our own "sandbox." We need a place to test out our bot (without annoying our classmates!) so go ahead and create a channel with your name in it, like `mikes-bot` or `marks-bot`.

After creating your channel, go to that channel and invite your bot to the channel. You can do it by typing `/invite` followed by your bot's name. For example, if I wanted to invite `@newsbot` to my channel, I'd type:

```
/invite @newsbot
```

After doing this, you should see a note saying: `Your bot was added to [your channel] by [you].`

<img src="https://raw.githubusercontent.com/computationaljournalism/columbia2020/master/images/bot_invite.png">

**Great!!** 🎉🎉

We're almost there! Now, let's install the Slack python library which makes it easy to post message to slack. To install Slack's python library, run the following:

In [ ]:
!pip install slackclient

### Sending our first message to Slack

To send a message from our bot (notebook) to Slack, you can update a few variables below and fire away. You'll need to:

1. add your API token/key, which you copied above, to the `SLACK_API_TOKEN` variable,
2. update the `channel` variable with the name of the channel that you just created. Make sure that it starts with `#`. For example, `#mike-test`.
3. feel free to update the `slack_message` variable - this is what your bot will post to your channel.



In [ ]:
import slack

# our slack api token
# copy this from the OAuth & Permissions --> "Bot User OAuth Access Token"
SLACK_API_TOKEN = ""

# initialize our slack api client
slack_client = slack.WebClient(token=SLACK_API_TOKEN)

# put your slack channel here, like #mike-test
channel = ""
slack_message = "Hello!"

# post a message to a channel
response = slack_client.chat_postMessage(channel=channel, text=slack_message)

print(response)

### Do you get an error?

If you see an error that says something like `The request to the Slack API failed`, you may not have completed your bot setup properly. 

```
SlackApiError: The request to the Slack API failed.
The server responded with: {'ok': False, 'error': 'missing_scope', 'needed': 'chat:write:bot', 'provided': 'calls:write'}
```


Most likely, you'll see an error message that says the following:

```
RuntimeError: This event loop is already running
```

To make this go away, we're going to add a bit of code to get around this error. Let's install this library first:

In [ ]:
# https://github.com/erdewit/nest_asyncio
!pip install nest_asyncio

In [ ]:
import slack

# to play nicely with Jupyter notebook
import asyncio
import nest_asyncio

nest_asyncio.apply()
# end of playing nice

# initialize our slack api client
slack_client = slack.WebClient(token=SLACK_API_TOKEN)

# put your slack channel here, like #mike-test
channel = ""
slack_message = "Hello!"

# post a message to a channel
response = slack_client.chat_postMessage(channel=channel, text=slack_message)

print(response)

### Great! Now, let's make something a bit more useful...

Let's imagine we want to have our newsbot send us top headlines a few times a day...alertings us when the News API has a new top headline. 

To do this, how might we have our bot run forever, waking up every so often? The following code will loop "forever", pausing for 5 seconds each time it runs:

In [ ]:
import time

# a loop that runs "forever"
while True:

    print("hello")

    # sleep for 5 seconds
    time.sleep(5)

To stop this, make sure you click the Stop icon `[]` in the notebook menu.

**Detour on while loops**

We have already seen `for` loops that iterate over a fixed set of items, say elements in a list. A `while` loop continues executing until the condition its testing for is no longer true. To make this real, let's consider tossing coins. We can simulate a coin toss with the `choice()` function in the `random` package.

In [ ]:
from random import choice

# a coin is just a list with two options - heads or tails
coin = ["Heads","Tails"]

# a toss is a random selection between these choices
toss = choice(coin)

# have a look -- do this several times
toss

Now, we can use the `while` loop toss a coin until we see five heads in total. We'll stop once we see the fifth head. Here's a simple loop.

In [ ]:
total_tosses = 0
total_heads = 0

while total_heads < 5:
    
    # keep track of the number of times we toss the coin
    total_tosses += 1
    
    # toss it
    toss = choice(coin)
    
    # test if its heads, and if so, increment the number of heads we've seen
    
    if toss == "Heads":
        total_heads += 1
        
# print out how many tosses we needed to see 5 heads

print(total_tosses)

**Back to our bot**

Ok, let's expand on that a bit and post a message to our Slack channel every few seconds. We're introducing a new python library here called the [datetime](https://docs.python.org/3/library/datetime.html) library. This is a very powerful library for dealing with, well, dates and times. We simply using it to print out the current time ("now").

In [ ]:
from datetime import datetime

# we're assuming you set the channel variable in a cell above
#channel = "#mike-test"

while True:

    # create a message that says "hello, the time is now 2020-03-11 12:34:00"
    slack_message = "hello, the time is now " + str(datetime.now())

    # post the message to our Slack channel
    response = slack_client.chat_postMessage(channel=channel, text=slack_message)

    print(response)
    
    # sleep for 5 seconds
    time.sleep(5)

Again, make sure you hit the Stop button above to have your bot stop spamming you!


Let's update our simple bot just a bit and post the top headline from the News API each time it runs:

In [ ]:
# let's call the news api for the top headlines
newsapi = NewsApiClient(api_key=NEWSAPI_KEY)

# let's loop forever
while True:

    # get the top headline from the news api
    top_headlines = newsapi.get_top_headlines(language='en', country='us')

    # get the first headline in the list
    top_headline = top_headlines['articles'][0]

    # build a slack message with the title and url
    slack_message = top_headline['title'] + ' ' + top_headline['url']

    # post the headline to our slack channel
    response = slack_client.chat_postMessage(channel=channel, text=slack_message)

    # sleep for 5 seconds
    time.sleep(5)
    

Well, that's not quite right! 🙃 We don't want out bot spamming us with the same headline every 5 seconds! What we'd really like is for our bot to post a headline to Slack only when it changes (i.e. when there is a new top headline). What's a technique that we can use to make sure we don't send the message over and over again?

In [ ]:
# let's modify the code to only post a message to slack when we have a new message

newsapi = NewsApiClient(api_key=NEWSAPI_KEY)

# keep track of the last/previous message we sent to slack
# we will initialize this to an empty string
previous_message = ""

while True:

    # get the top headline from the news api
    top_headlines = newsapi.get_top_headlines(language='en', country='us')

    top_headline = top_headlines['articles'][0]

    # build a slack message with the title and url
    slack_message = top_headline['title'] + ' ' + top_headline['url']

    # have we sent this one before?
    if slack_message == previous_message:
        print('we have already sent this message: ' + slack_message)
    else:
        # it's a new article! let's post it to our slack channel
        response = slack_client.chat_postMessage(channel=channel, text=slack_message)

        # save the message we just sent as our "previous" (or last) message
        previous_message = slack_message
    
    # sleep for 5 seconds
    time.sleep(5)
    

**Notice** that we still have a problem with sending the same messages if we stop and start out bot. How might we fix that? Where can we keep track of the previous messages that we've sent? Something that would be persistant in-between the bot stopping and starting?

### End of Part 1

* Part 2 will be about setting up Slack Events, flask, etc
* Part 3 will be about building the cornoa-bot functionality (using Slack Events)